In [1]:
import pandas as pd
import json
import datetime as dt

In [2]:
# Take: Each activity has active duration, duration, originalDuration. These will be columns, not sure if they are ever different. If not, just use one.
# coumns from under activityLevel value array that contains same info as above, just need to see if the same.
# Take: activityLevel_sedentary, activityLevel_lightly, activityLevel_fairly, activityLevel_very, each containing minutes
# Take: activityName, activityTypeId, averageHeartRate, calories, 
# from heartRateZones: 
# Take: hrZone_OOR_mins, hrZone_Fatburn_mins, hrZone_Cardio_mins, hrZone_Peak_mins /// hrZone_OOR_cals, hrZone_Fatburn_cals, hrZone_Cardio_cals, hrZone_Peak_cals
# Take: logID, originalStartTime, startTime, steps

# Activity Data Cleaning

In [4]:
# Loop through activity files normalize, pd them, drop cols, drop more cols (might need a catch), concat all dfs ,
# create column names, loop through and transform data, drop more columns, change types

# Sleep Data Cleaning

In [5]:
# Loop through sleep logs, keep only certain columns per each, concat all dfs together

In [6]:
# Read sleep csv, (go through date conversion stuff and only keep necessary things), 
# (dont drop first row, idk why that is there), create datetimekey col and convert to datetime..., perform join, 

# Stress Data Cleaning

In [8]:
# Perform any necessary data cleaning as was done on sleep data csv...

# Skin / HRV / BRV Data Cleaning

In [7]:
#Copy work dne in previous notebook, try to  find a way to auto detect the # of files with skin temp...